In [1]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [2]:
import coralme
from coralme.builder.main import MEBuilder, MEReconstruction, METroubleshooter
from coralme.builder.curation import MECurator, MEManualCuration


/home/jt/UCSD/coralme/coralme/builder/main.py:30: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")
/home/jt/UCSD/coralme/coralme/builder/preprocess_inputs.py:15: UserWarning: This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.
  warnings.warn("This pandas version does not allow for correct warning handling. Pandas >=1.5.1 is suggested.")


In [3]:
org = 'bacillus'
organism = './{:s}/organism.json'.format(org)
inputs = './{:s}/input.json'.format(org)

### Full

In [ ]:
builder = MEBuilder(*[organism, inputs])
builder.configuration['run_bbh_blast'] = False
# builder.configuration['dev_reference'] = Falsehttps://pubmed.ncbi.nlm.nih.gov/15351641/

In [ ]:
builder.generate_files(overwrite=True)
# builder.save_builder_info()

In [ ]:
# builder.org = coralme.builder.organism.Organism(builder.configuration,False)
# builder.org.get_organism()

In [ ]:
builder.build_me_model(overwrite=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Quick

In [ ]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])

In [ ]:
builder.build_me_model(overwrite=False, update=True, prune=False)

In [ ]:
builder.troubleshoot(growth_key_and_value = { builder.me_model.mu : 0.001 })

### Load builder

In [4]:
builder = MEBuilder(*['./{:s}/coralme-config.yaml'.format(org)])
builder.me_model = builder.load(builder.configuration['out_directory']+'MEModel-step2-{}.pkl'.format(org))
me = builder.me_model

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-12
Read LP format model from file /tmp/tmpwab61_ar.lp
Reading time = 0.00 seconds
: 0 rows, 0 columns, 0 nonzeros
Read LP format model from file /tmp/tmpqyslshgr.lp
Reading time = 0.01 seconds
: 998 rows, 2524 columns, 10562 nonzeros
Read LP format model from file /tmp/tmppr2tanvd.lp
Reading time = 0.01 seconds
: 1088 rows, 2554 columns, 10544 nonzeros


In [5]:
me.optimize(precision='dq')

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Optimal
        5	0.2634427628817360	Not feasible
        6	0.2195356357347800	Not feasible
        7	0.1975820721613020	Not feasible
        8	0.1866052903745630	Not feasible
        9	0.1811168994811935	Not feasible
       10	0.1783727040345087	Not feasible
       11	0.1770006063111663	Not feasible
       12	0.1763145574494951	Optimal
       13	0.1766575818803308	Optimal
       14	0.1768290940957485	Optimal
       15	0.1769148502034574	Not feasible
       16	0.1768719721496030	Optimal
       17	0.1768934111765302	Optimal
       18	0.1769041306899938	Not feasible
       19	0.1768987709332620	Optimal
       20	0.1769014508116279	Not feasible
       21	0.1769001108724449	Not feasible
       22	0.1768994409028535	Not feasible


True

In [8]:
for r in me.reactions:
    lb,ub = r.bounds
    lb = lb.subs(me.mu,me.solution.fluxes["biomass_dilution"]) if hasattr(lb,'subs') else lb
    ub = ub.subs(me.mu,me.solution.fluxes["biomass_dilution"]) if hasattr(ub,'subs') else ub
    if me.solution.fluxes[r.id] < lb or me.solution.fluxes[r.id] > ub:
        print(r)
        print(lb,ub)
        print(me.solution.fluxes[r.id])
        print()

M1PD_FWD_BSU03990-MONOMER: [2.81856022570259e-6*mu1] BSU03990-MONOMER + 1.0 mnl1p_c + 1.0 nad_c --> 1.0 f6p_c + 1.0 h_c + 1.0 nadh_c
0 999999.0
-3.2321967849547743e-28

APNPT_REV_BSU23760-MONOMER_mod_mg2(1): [2.77531548994142e-5*mu1] BSU23760-MONOMER_mod_mg2(1) + 1.0 adp_c + 1.0 h_c + 1.0 pan4p_c --> 1.0 atp_c + 1.0 ptth_c
0 999999.0
-1.1556741177480996e-18

APNPT_FWD_BSU23760-MONOMER_mod_mg2(1): [2.77531548994142e-5*mu1] BSU23760-MONOMER_mod_mg2(1) + 1.0 atp_c + 1.0 ptth_c --> 1.0 adp_c + 1.0 h_c + 1.0 pan4p_c
0 999999.0
-1.1556741177480996e-18

DNA_replication: 0.28242492549115594 datp_c + 0.21757507450884406 dctp_c + 0.21757507450884406 dgtp_c + 0.28242492549115594 dttp_c --> 0.3079068982281192 DNA_biomass + 1.0 ppi_c
0.0338606118945269 0.0338606118945269
0.03386061189452694



### Find issue

In [ ]:
builder.find_issue('GMET_RS15370')

### Curation notes

In [ ]:
builder.curation_notes